# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/output_data/weather_data.csv")
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ribeira grande,78,PT,1580271745,94,38.52,-28.70,60.12,19.71
1,1,dikson,100,RU,1580271746,95,73.51,80.55,-18.06,14.38
2,2,bengkulu,100,ID,1580271514,94,-3.80,102.27,72.45,13.62
3,4,port alfred,60,ZA,1580271516,83,-33.59,26.89,68.00,4.00
4,5,cortez,1,US,1580271746,79,37.35,-108.59,26.60,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_df[['Lat', 'Lng']]
weight = vacation_df['Humidity'].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temp = vacation_df.loc[(vacation_df['Max Temp'] <80) & (vacation_df['Max Temp'] >70)]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed'] <10]
ideal_cloud = ideal_wind.loc[ideal_wind['Cloudiness']==0] 
ideal_cloud

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
64,69,paraguacu paulista,0,BR,1580271857,67,-22.41,-50.58,72.28,3.91
221,249,el triunfo,0,HN,1580272050,86,13.12,-87.00,73.18,2.57
281,318,rio grande,0,BR,1580272082,87,-32.03,-52.10,72.00,1.01
321,364,san patricio,0,MX,1580272151,63,19.22,-104.70,76.05,4.29
400,447,champerico,0,GT,1580272205,82,14.30,-91.92,72.86,4.03
439,490,monrovia,0,LR,1580272193,87,6.30,-10.80,79.41,4.43
527,586,veraval,0,IN,1580272350,40,20.90,70.37,72.00,8.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(ideal_cloud, columns=['City', 'Cloudiness', 'Country', 'Date',
                                             'Humidity', 'Lat', 'Lng', 'Max Temp',
                                             'Wind Speed'])
hotel_df['Hotel Name']=''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
64,paraguacu paulista,0,BR,1580271857,67,-22.41,-50.58,72.28,3.91,
221,el triunfo,0,HN,1580272050,86,13.12,-87.00,73.18,2.57,
281,rio grande,0,BR,1580272082,87,-32.03,-52.10,72.00,1.01,
321,san patricio,0,MX,1580272151,63,19.22,-104.70,76.05,4.29,
400,champerico,0,GT,1580272205,82,14.30,-91.92,72.86,4.03,
439,monrovia,0,LR,1580272193,87,6.30,-10.80,79.41,4.43,
527,veraval,0,IN,1580272350,40,20.90,70.37,72.00,8.72,


In [7]:
params = {"radius": 5000,
          "types": "restaurant",
          "key": g_key
         }

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    
    params['location'] = f"{lat},{lon}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    names = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = names["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing result... skipping.")

In [8]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
64,paraguacu paulista,0,BR,1580271857,67,-22.41,-50.58,72.28,3.91,Suspiro Restaurante & Presentes
221,el triunfo,0,HN,1580272050,86,13.12,-87.00,73.18,2.57,Golosinas Y Pulperia La Chela
281,rio grande,0,BR,1580272082,87,-32.03,-52.10,72.00,1.01,Pandorato
321,san patricio,0,MX,1580272151,63,19.22,-104.70,76.05,4.29,Señor Froy´s
400,champerico,0,GT,1580272205,82,14.30,-91.92,72.86,4.03,7 Seas Restaurant
439,monrovia,0,LR,1580272193,87,6.30,-10.80,79.41,4.43,Beirut
527,veraval,0,IN,1580272350,40,20.90,70.37,72.00,8.72,Lords Inn Somnath


In [10]:
hotel_df.to_csv('output_data/Hotels.csv', index=False)

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
#creating marker locations
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))